# Numpy Array Operations:  Axes and Broadcasting 

There is an excellent introduction to `numpy` multi-dimensional arrays on the [scipy](https://docs.scipy.org/doc/numpy-dev/user/quickstart.html) website.  In this note, we cover two concepts in a little more detail:
* Using the `axis` feature 
* Python broadcasting

We will need both of these for performing many of the numerical operations for the ML class.

As usual, we begin by loading the `numpy` package.

In [33]:
import numpy as np

## Axis Parameter

Many operations in the `numpy` package can take an optional `axis` parameter to specify which dimensions the operation is to be applied.  This is extremely useful for multi-dimensional data.  To illustrate the `axis` parameter, consider a matrix the `(3,2)` array `X` defined as:

In [3]:
X = np.arange(6).reshape(3,2)
print(X)

[[0 1]
 [2 3]
 [4 5]]


An operation like `np.mean` or `np.sum` takes the mean or sum of *all* elements in the array. 

In [4]:
print(np.mean(X))
print(np.sum(X))

2.5
15


To take only the `sum` along each column, we can use the `axis` parameter.

In [5]:
print(np.sum(X,axis=0))

[6 9]


Since `X` has shape `(3,2)`, the output `np.sum(X,axis=0)` is of shape `(2,)`.  Similarly, we can take the `sum` along each row:

In [6]:
print(np.sum(X,axis=1))

[1 5 9]


You can apply this to higher-order arrays:

In [7]:
X = np.arange(24).reshape(2,3,4)  # shape = (2,3,4)
print(X)
print('\n')
Y1 = np.sum(X,axis=0)             # shape = (3,4)
Y2 = np.sum(X,axis=1)             # shape = (2,4)
print('Y1 = ')
print(Y1)
print('Y2 = ')
print(Y2)

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]


Y1 = 
[[12 14 16 18]
 [20 22 24 26]
 [28 30 32 34]]
Y2 = 
[[12 15 18 21]
 [48 51 54 57]]


## Broadcasting

**Broadcasting** is a powerful tool in Python for performing operations on matrices that we will use throughout the ML class.  A good tutorial on broadcasting can be found on the [scipy broadcasting page](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html).  Here we provide some examples.   

### Example 1:  Mean Removal

Suppose that `X` is a data matrix of shape `(n,p)`.  That is, there are `n` data points and `p` features per point.  Often, we have to remove the mean from each feature.  That is, we want to compute the mean for each feature and then remove the mean from each column.  We could do this with a for-loop as:
   
    Xm = np.zeros(p)      # Mean for each feature
    X1_demean = np.zeros((n,p))  # Transformed features with the means removed
    for j in range(p):
       Xm[j] = np.mean(X[:,j])
       for i in range(n):
           X_demean[i,j] = X[i,j] - Xm[j]
           
The code below does this without a for loop using the `axis` parameter and broadcasting.

In [41]:
# Generate some random data
n = 100
p = 5
X = np.random.rand(n,p)
print('\n')

# Compute the mean per column using the axis command
Xm = np.mean(X,axis=0)  # This is a p-dim matrix

# Subtract the mean
X_demean = X - Xm[None,:]


The command `Xm = np.mean(X,axis=0)` computes the mean of each column which is a `p` dimensional array.  Then, `Xm[None,:]` converts this to a `(1,p)` shape array.  Using Python broadcasting, we can then subtract the `Xm[None,:]` from `X`.

How does this work? First, we use `None` keyword to reshape the matrices `X` and `Y` to compatible sizes

     X[:,None,:]    # Shape nx,  1, p
     Y[None,:,:]    # Shape 1,  ny, p
     
The two matrices can be subtracted so that

     DXY[i,j,k]  = X[i,k] - Y[j,k]
     
Then, `d[i,j] = sum_k (X[i,k] - Y[j,k])**2`, which is the norm squared of the vector differences.

### Example 4:  Outer product

The *outer product* of vectors `x` and `y` is the matrix `Z[i,j] = x[i]y[j]`.  This can be performed in one line as follows

In [44]:
# Some random data
nx = 100
ny = 10
x = np.random.rand(nx)

y = np.random.rand(ny)
print(y)
print('\n')
print(y[None,:])
print('\n')

# Compute the outer product in one line
Z = x[:,None]*y[None,:]
print(Z)


[0.61984375 0.23516808 0.32858326 0.76249426 0.45678753 0.32319973
 0.39492751 0.8982788  0.5780751  0.63415698]


[[0.61984375 0.23516808 0.32858326 0.76249426 0.45678753 0.32319973
  0.39492751 0.8982788  0.5780751  0.63415698]]


[[3.59775260e-01 1.36498362e-01 1.90719239e-01 4.42573746e-01
  2.65132710e-01 1.87594484e-01 2.29227361e-01 5.21387022e-01
  3.35531526e-01 3.68083076e-01]
 [3.60502646e-02 1.36774332e-02 1.91104831e-02 4.43468532e-02
  2.65668750e-02 1.87973758e-02 2.29690808e-02 5.22441152e-02
  3.36209897e-02 3.68827258e-02]
 [6.16648667e-01 2.33955868e-01 3.26889527e-01 7.58563862e-01
  4.54432949e-01 3.21533749e-01 3.92891793e-01 8.93648476e-01
  5.75095321e-01 6.30888122e-01]
 [4.05524590e-01 1.53855611e-01 2.14971261e-01 4.98851802e-01
  2.98847212e-01 2.11449159e-01 2.58376109e-01 5.87687042e-01
  3.78198002e-01 4.14888835e-01]
 [9.50413276e-02 3.60585816e-02 5.03820349e-02 1.16914088e-01
  7.00397374e-02 4.95565725e-02 6.05546717e-02 1.37734081e-01
  8.86368944e-0

Here:

     x[:,None] # Has shape (nx,  1)
     y[None,:] # Has shape ( 1, ny)
     
So, with python broadcasting:

     Z = x[:,None]*y[None,:] # has shape (nx,  ny)


**Exercise 1:**  Given a matrix `X`, compute the matrix `Y`, where the rows of `X` are normalized to one.  That is:

     Y[i,j] = X[i,j] / sum_j X[i,j]   

In [57]:
X = np.random.rand(4,3)
print(X)
row = X.sum(axis=1)
print(row)
Y = X / X.sum(axis=1)
print(Y)

[[0.9435497  0.83575238 0.33412394]
 [0.87040377 0.03392012 0.17247606]
 [0.82746239 0.28797803 0.11051434]
 [0.03772937 0.07416283 0.37979263]]
[2.11342602 1.07679995 1.22595476 0.49168484]


ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

**Exercise 2:** Diagonal multiplication.  Given a matrix `X` and a vector `d`, compute `Y = diag(d)*X`.

In [45]:
X = np.random.rand(5,3)
d = np.random.rand(5)
# Y = diag(d)*X

NameError: name 'diag' is not defined